In [1]:
import csv
import math
import pandas as pd
import random
import gzip
import torch
from sklearn import metrics
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import warnings
import torch.nn.functional as F
from torch import optim
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import make_scorer,balanced_accuracy_score,roc_curve,precision_recall_curve,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import acf
from numpy import hstack
import time
warnings.filterwarnings("ignore")

c:\anaconda\envs\Deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
class datasets():
    def __init__(self,data,n_steps_in):
        self.data=data
        self.n_steps_in=n_steps_in
        self.n0 = np.where(data['PeriodsSepLastTwoNnZeroDemands']>0)[0][0]
        self.n1 = self.n0+int((data.shape[0]-self.n0)*90/100)
        self.size=int((self.n1)/5)
        in_seq1=data['ZNZDemand'].values[self.n0:].reshape((-1, 1))
        in_seq2=data['LastQty'].values[self.n0:].reshape((-1, 1))
        in_seq3=data['WeekDay'].values[self.n0:].reshape((-1, 1))
        in_seq4=data['Interval'].values[self.n0:].reshape((-1, 1))
        in_seq5=data['PeriodsSepLastTwoNnZeroDemands'].values[self.n0:].reshape((-1, 1))
        in_seq6=data['Month'].values[self.n0:].reshape((-1, 1))
        in_seq7=data['Qty'].values[self.n0:].reshape((-1, 1))
        self.dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4,in_seq5,in_seq6,in_seq7))
        if np.count_nonzero(np.isnan(self.dataset))>0:
            print('nan found')
        self.scaler = MinMaxScaler(feature_range = (0, 1))
        self.scaler.fit(data[['Qty']].values[self.n0:self.n1])
        #random.shuffle(self.dataset)
        
        
    def get_deep2net_datasets(self):
        X = []
        for i in range(len(self.dataset)):
            # find the end of this pattern
            end_ix = i + self.n_steps_in
            out_end_ix = end_ix + 1
            # check if we are beyond the dataset
            if (out_end_ix-1) > len(self.dataset):
                break
            # gather input and output parts of the pattern
            seq_x1, seq_y1 = list(self.dataset[i:end_ix,1:-1]), list(self.dataset[end_ix-1:out_end_ix-1, 0])
            seq_x2, seq_y2 = [self.dataset[i+self.n_steps_in-1:i+self.n_steps_in,1],self.dataset[i+self.n_steps_in-1:i+self.n_steps_in,3]], list(self.dataset[i+self.n_steps_in-1:i+self.n_steps_in,6])
            X.append([seq_x1, seq_y1, seq_x2, seq_y2])
        train_data=X[:self.n1]
        test_data=X[self.n1:]
        calib_data = X[:4*self.size]
        valid_data = X[4*self.size:self.n1]
        #random.shuffle(calib_data)
        #random.shuffle(valid_data)
        #random.shuffle(train_data)
        #random.shuffle(test_data)
        return calib_data,valid_data,train_data,test_data
    
    def get_reg_datasets(self):
        X = []
        dataset1=self.dataset.copy()
        dataset1[:,-1]=self.scaler.transform(np.array(dataset1[:,-1]).reshape(len(dataset1[:,-1]),1)).ravel()
        for i in range(self.n_steps_in-1,len(dataset1)):
            # gather input and output parts of the pattern
            seq_x, seq_y = dataset1[i:i+1,1:-1], dataset1[i:i+1, -1]
            X.append([seq_x,seq_y])
        train_data=X[:self.n1]
        test_data=X[self.n1:]
        calib_data = X[:4*self.size]
        valid_data = X[4*self.size:self.n1]
        return calib_data,valid_data,train_data,test_data,self.scaler
    
    def get_ADI(self):
        ADI = (self.data['ZNZDemand'].value_counts()[1]+self.data['ZNZDemand'].value_counts()[0])/self.data['ZNZDemand'].value_counts()[1]
        return(ADI)
    
    def get_n0(self):
        return self.n0+self.n_steps_in
    
    def get_n1(self):
        return self.n1+self.n0+self.n_steps_in-1
    
    def get_CV2(self):
        Test_df = (self.data[self.data['ZNZDemand']==1][['Qty','Interval']]).dropna(axis=0)
        CV2 = (Test_df['Qty'].std()/Test_df['Qty'].mean())**2
        return (CV2)
    
    def get_n_obs(self):
        n_obs=self.data.shape[0]-self.n0
        return (n_obs)
    
    def get_train_Y(self):
        return (self.dataset[:self.n1, -1].tolist())
    
    def get_test_Y(self):
        return (self.dataset[self.n1+self.n_steps_in-1:, -1].tolist())
    
    
    def get_Nlags(self):
        autocorr=acf(self.data['Qty'],False,100)
        Nlags=np.where(autocorr==max(autocorr[(autocorr<1)]))[0][0]
        return (Nlags)
    
    def get_w0(self):
        w0 = 1
        return w0
    
    def get_w1(self):
        w1 = self.data['ZNZDemand'].value_counts()[0]/self.data['ZNZDemand'].value_counts()[1]/2
        return (w1)

In [3]:
class dataset_load(Dataset):
    def __init__(self,xy=None):
        self.x1_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y1_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x2_data=np.asarray([el[2] for el in xy],dtype=np.float32).squeeze(axis=2)
        self.y2_data =np.asarray([el[3] for el in xy ],dtype=np.float32)
        self.x1_data = torch.from_numpy(self.x1_data)
        self.y1_data = torch.from_numpy(self.y1_data)
        self.x2_data = torch.from_numpy(self.x2_data)
        self.y2_data = torch.from_numpy(self.y2_data)
        self.len=len(self.x1_data)
    def __getitem__(self, index):
        return self.x1_data[index], self.y1_data[index], self.x2_data[index], self.y2_data[index]
    def __len__(self):
        return self.len

In [4]:
class dataset_load1(Dataset):
    def __init__(self,xy=None):
        self.x=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)
        self.len=len(self.x)
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [5]:
def logsampler(a,b):
    x=np.random.uniform(low=0,high=1)
    y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
    return y

In [6]:
def MASE(X,Y,F):
    N1=len(X)
    N2=len(Y)
    D1=np.sum(abs(np.array(Y)-np.array(F)))/N2
    D2=np.sum(abs(np.array(X[1:])-np.array(X[:-1])))/N1
    return (D1/D2)

In [7]:
def RMSSE(X,Y,F):
    h=len(Y)
    n=len(X)
    D1=(1/h)*np.sum((np.array(Y)-np.array(F))**2)
    D2=(1/(n-1))*np.sum((np.array(X[1:])-np.array(X[:-1]))**2)
    return (np.sqrt(D1/D2))

# Network 1

In [8]:
class Deepnet(nn.Module):
    def __init__ (self,RNN,RNN_hidden_size,RNN_mean,RNN_sigma,layer_size,dropprob,n_features,Nout=1):
        super(Deepnet,self).__init__()
        self.RNN=RNN
        self.RNN_hidden_size=RNN_hidden_size
        self.RNN_sigma=RNN_sigma   
        self.layer_size=layer_size
        self.RNN_mean=RNN_mean   
        self.input_channels=n_features
        self.dropprob=dropprob
        self.out=Nout
        if self.RNN=='LSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiLSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        elif self.RNN=='GRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiGRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        for layer_p in self.rnn._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    torch.nn.init.normal_(self.rnn.__getattr__(p),mean=RNN_mean,std=RNN_sigma)  
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
        #weights between LSTM or GRU layers and fully connected layer
        self.classifier = nn.Sequential(
            nn.Linear(self.FC_size,self.layer_size),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropprob, inplace=False),
            nn.Linear(self.layer_size, self.out),
            nn.Sigmoid())
    
    def forward(self,x):
        x=x.permute(1,0,2)
        output,_=self.rnn(x)
        if self.RNN=='BiLSTM' or self.RNN=='BiGRU':
            Normal_RNN=output[-1, :, :self.RNN_hidden_size]
            Rev_RNN=output[0, :, self.RNN_hidden_size:]
            x = torch.cat((Normal_RNN, Rev_RNN), 1)
            x=self.dropout(x)
        else:
            x = output[-1, :, :]
            x=self.dropout(x)
        x = self.classifier(x)
        return (x)

In [9]:
def Calibration(RNN,w1,w0,calib_loader,valid_loader,n_features,metric):
    best_AUC = 0
    if verbose:
        print('Training on ',device)
    max_learning_steps = 500
    RNN_hidden_size_list = [20, 50, 80, 100]
    dropoutList = [0, 0.15, 0.3, 0.45]
    layer_size_list = [32, 64]
    learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
    max_num_models = 40
    for number in range(max_num_models):
        if verbose:
            print('model {0} out of {1}'.format(number+1,max_num_models))
        # hyper-parameters
        RNN_hidden_size = random.choice(RNN_hidden_size_list)
        dropprob = random.choice(dropoutList)
        layer_size = random.choice(layer_size_list)
        learning_rate=random.choice(learning_rate_list)
        RNN_mean = random.choice([-1,1])*logsampler(10 ** -4, 1)
        RNN_sigma = logsampler(10 ** -4, 10 ** -2)
        model = Deepnet(RNN,RNN_hidden_size,RNN_mean,RNN_sigma,layer_size,dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
        learning_steps = 0
        while learning_steps < max_learning_steps:
            loss_per_epoch = 0
            model.train()
            for batch_idx, (data1, target1, _, _) in enumerate(calib_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                outputs = model(data)
                criterion = nn.BCELoss(reduction='mean',weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))
                loss = criterion(outputs, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_per_epoch+=loss.item()
            learning_steps+=1
            if (learning_steps % 100 == 0):
                if verbose:
                    print("model trained for {0} epochs, loss per epoch is {1}.".format(learning_steps,loss_per_epoch/(batch_idx+1)))
                with torch.no_grad():
                    model.eval()
                    auc = []
                    for batch_idx, (data1, target1, _, _) in enumerate(valid_loader):
                        data = data1.to(device)
                        target = target1.to(device)
                        # Forward pass
                        output = model(data)
                        pred = output.cpu().detach().numpy().reshape(output.shape[0])
                        labels = target.cpu().numpy().reshape(output.shape[0])
                        if output.shape[0] > 60:
                            if (metric=='ROC'):
                                auc.append(metrics.roc_auc_score(labels, pred))
                            elif (metric=='PRC'):
                                precision, recall, _ = precision_recall_curve(labels, pred)
                                auc.append(metrics.auc(recall, precision))
                            else :
                                print('Choose proper metric')
                    AUC = np.mean(auc)
                    # print(AUC)
                    if AUC > best_AUC:
                        best_AUC = AUC
                        best_learning_steps = learning_steps
                        best_LearningRate = learning_rate
                        best_RNN_hidden_size=RNN_hidden_size
                        best_dropprob = dropprob
                        best_layer_size= layer_size
                        best_RNN_sigma = RNN_sigma
                        best_RNN_mean = RNN_mean
    if verbose:
        print('best_AUC=', best_AUC)
        print('best_learning_steps=', best_learning_steps)
        print('best_LearningRate=', best_LearningRate)
        print('best_dropprob=', best_dropprob)
        print('best_RNN_hidden_size=', best_RNN_hidden_size)
        print('best_layer_size=', best_layer_size)
        print('best_RNN_sigma=', best_RNN_sigma)
        print('best_RNN_mean=', best_RNN_mean)
    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_RNN_hidden_size': best_RNN_hidden_size,
                            'best_layer_size': best_layer_size, 
                            'best_RNN_sigma': best_RNN_sigma,
                            'best_RNN_mean':best_RNN_mean}
    return best_hyperparameters

In [10]:
def Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric):
    best_learning_steps=8#best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_RNN_mean=best_hyperparameters['best_RNN_mean']
    best_RNN_sigma=best_hyperparameters['best_RNN_sigma']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_AUC=0
    best_threshold=0.5
    for number_models in range(5):
        model = Deepnet(RNN,best_RNN_hidden_size,best_RNN_mean,best_RNN_sigma,best_layer_size,best_dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=best_LearningRate)
        learning_steps=0
        model.train()
        while learning_steps<best_learning_steps:
            for batch_idx, (data1, target1, _, _) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                outputs = model(data)
                criterion = nn.BCELoss(reduction='mean',weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))
                loss = criterion(outputs, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            learning_steps+=1
        with torch.no_grad():
            model.eval()
            auc=[]
            threshold=[]
            for batch_idx, (data1, target1, _, _) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                output = model(data)
                pred = output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if pred.shape[0]>60:
                    if (metric=='ROC'):
                        auc.append(metrics.roc_auc_score(labels, pred))
                        fpr, tpr, thresholds = roc_curve(labels, pred)
                        gmeans=np.sqrt(tpr*(1-fpr))
                        threshold.append(thresholds[np.argmax(gmeans)])
                    elif (metric=='PRC'):
                        precision, recall, thresholds = precision_recall_curve(labels, pred)
                        auc.append(metrics.auc(recall, precision))
                        gmeans=(2*precision*recall)/(precision+recall)
                        threshold.append(thresholds[np.argmax(gmeans)])
                    else :
                        print('Choose proper metric') 
            AUC_training=np.mean(auc)
            if verbose:
                print('AUC on training data for model ',number_models+1,' = ',AUC_training)
            if AUC_training>best_AUC:
                best_AUC=AUC_training
                best_threshold=np.mean(np.array(threshold),axis=0)
                best_model=model
    return best_model,best_threshold

In [11]:
def test_predict(best_model,test_loader):
    with torch.no_grad():
        best_model.eval()
        auc = []
        for batch_idx, (data1, target1, _, _) in enumerate(test_loader):
            data = data1.to(device)
            target = target1.to(device)
            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            labels = target.cpu().numpy().reshape(output.shape[0])
            if output.shape[0] > 50:
                auc.append(metrics.roc_auc_score(labels, pred))
        AUC_test = np.mean(auc)
        if verbose:
            print('AUC on test data is ',AUC_test)
    return (AUC_test)

# Network 2

In [12]:
class Deepnet2(nn.Module):
    def __init__ (self,layer_number,layer_size,dropprob,n_features):
        super(Deepnet2,self).__init__()
                
        self.layer_size=layer_size
        self.layer_number=layer_number
        self.input_channels=n_features
        self.dropprob=dropprob
        

        
        self.FC=self.layer_size
        self.fc1=nn.Linear(self.input_channels,self.layer_size)
        if (self.layer_number>1):
            self.fc2=nn.Linear(self.layer_size,self.layer_size)
        if (self.layer_number>2):
            self.fc3=nn.Linear(self.layer_size,self.layer_size)
        self.fc_f=nn.Linear(self.layer_size,1)
        
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        if (self.layer_number>1):
            x=F.relu(self.fc2(x))
            x=self.dropout(x)
        
        if (self.layer_number>2):
            x=F.relu(self.fc3(x))
            x=self.dropout(x)
        
        x=F.relu(self.fc_f(x))

        return (x)

In [13]:
def Calibration2(calib_loader,valid_loader,n_features,best_model1,best_threshold):
    best_MSE = 100
    if verbose:
        print('Training on ',device)
    max_learning_steps = 500
    dropoutList = [0, 0.15, 0.3, 0.45]
    layer_number_list=[1,2,3]
    layer_size_list = [16, 32]
    learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
    max_num_models = 40
    for number in range(max_num_models):
        if verbose:
            print('model {0} out of {1}'.format(number+1,max_num_models))
        # hyper-parameters
        dropprob = random.choice(dropoutList)
        layer_number=random.choice(layer_number_list)
        layer_size = random.choice(layer_size_list)
        learning_rate=random.choice(learning_rate_list)
        model_MSE = []
        model = Deepnet2(layer_number,layer_size,dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

        learning_steps = 0
        while learning_steps <= max_learning_steps:
            loss_per_epoch = 0
            model.train()
            for batch_idx, (data1, target1, data2, target2) in enumerate(calib_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                ZNZ_output = best_model1(data).cpu().detach().numpy()
                ZNZ_output=np.where(ZNZ_output<best_threshold,0,ZNZ_output)
                ZNZ_output=np.where(ZNZ_output>best_threshold,1,ZNZ_output)
                data2 = torch.hstack((data2,torch.tensor(ZNZ_output)))
                data = data2.to(device)
                target = target2.to(device)
                # Forward pass
                output = model(data)
                criterion = nn.MSELoss()
                loss = criterion(output,target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_per_epoch+=loss.item()
            learning_steps += 1
            if learning_steps % 100 == 0:
                if verbose:
                    print("model trained for {0} epochs, loss per epoch is {1}.".format(learning_steps,loss_per_epoch/(batch_idx+1)))
                with torch.no_grad():
                    model.eval()
                    mse = []
                    for batch_idx, (data1, target1, data2, target2) in enumerate(valid_loader):
                        data = data1.to(device)
                        target = target1.to(device)
                        # Forward pass
                        ZNZ_output = best_model1(data).cpu().detach().numpy()
                        ZNZ_output=np.where(ZNZ_output<best_threshold,0,ZNZ_output)
                        ZNZ_output=np.where(ZNZ_output>best_threshold,1,ZNZ_output)
                        data2 = torch.hstack((data2,torch.tensor(ZNZ_output)))
                        data = data2.to(device)
                        target = target2.to(device)
                        # Forward pass
                        output = model(data)
                        pred = output.cpu().detach().numpy().reshape(output.shape[0])
                        labels = target.cpu().numpy().reshape(output.shape[0])
                        if output.shape[0] > 50:
                            mse.append(metrics.mean_squared_error(labels, pred))  
                    MSE=np.mean(mse)
                    if MSE < best_MSE:
                        best_MSE = MSE
                        best_learning_steps = learning_steps
                        best_LearningRate = learning_rate
                        best_dropprob = dropprob
                        best_layer_number= layer_number
                        best_layer_size= layer_size
    
    if verbose:
        print('best_MSE=', best_MSE)
        print('best_learning_steps=', best_learning_steps)
        print('best_LearningRate=', best_LearningRate)
        print('best_dropprob=', best_dropprob)
        print('best_layer_number=', best_layer_number)
        print('best_layer_size=', best_layer_size)

    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_layer_number': best_layer_number,
                            'best_layer_size': best_layer_size}
    return best_hyperparameters

In [14]:
def Train_model2(best_hyperparameters,train_loader,n_features,best_model1,best_threshold):
    best_learning_steps=best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_layer_number=best_hyperparameters['best_layer_number']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_MSE=100

    for number_models in range(5):
        model = Deepnet2(best_layer_number,best_layer_size,best_dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=best_LearningRate)

        learning_steps=0
        model.train()
        while learning_steps<=best_learning_steps:
            for batch_idx, (data1, target1, data2, target2) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                ZNZ_output = best_model1(data).cpu().detach().numpy()
                ZNZ_output=np.where(ZNZ_output<best_threshold,0,ZNZ_output)
                ZNZ_output=np.where(ZNZ_output>best_threshold,1,ZNZ_output)
                data2 = torch.hstack((data2,torch.tensor(ZNZ_output)))
                data = data2.to(device)
                target = target2.to(device)
                output = model(data)
                loss = nn.MSELoss()
                L=loss(output,target)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
            learning_steps+=1

        with torch.no_grad():
            model.eval()
            mse=[]
            for batch_idx, (data1, target1, data2, target2) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                ZNZ_output = best_model1(data).cpu().detach().numpy()
                ZNZ_output=np.where(ZNZ_output<best_threshold,0,ZNZ_output)
                ZNZ_output=np.where(ZNZ_output>best_threshold,1,ZNZ_output)
                data2 = torch.hstack((data2,torch.tensor(ZNZ_output)))
                data = data2.to(device)
                target = target2.to(device)
                # Forward pass
                output = model(data)
                pred=output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if output.shape[0]>60:
                    mse.append(metrics.mean_squared_error(labels, pred))
            MSE_Training=np.mean(mse)     
            if verbose:
                print('MSE on training data for model ',number_models+1,' = ',MSE_Training)
            if MSE_Training<best_MSE:
                best_MSE=MSE_Training
                best_model=model
    return best_model

In [15]:
def test_generate_data(data1,pred1,data2,pred2,step_idx):
    if pred1==0:
        lstqt = data1[step_idx][-1][0]
        weekday = data1[step_idx+1][-1][1]
        interv = data1[step_idx+1][-1][2]
        PSLTNZD = data1[step_idx+1][-1][3]
        Month = data1[step_idx+1][-1][-1]
    else:
        lstqt = pred2
        weekday = data1[step_idx+1][-1][1]
        interv = 1
        PSLTNZD = data1[step_idx][-1][2]
        Month = data1[step_idx+1][-1][-1]
    new_data1 = [lstqt,weekday,interv,PSLTNZD,Month]
    new_data2 = [lstqt,interv]
    data1[step_idx+1] = torch.vstack((data1[step_idx][1:],torch.tensor(new_data1).unsqueeze(dim=0)))
    data2[step_idx+1] = torch.tensor(new_data2)
    return (data1, data2)

In [16]:
def test_predict2(best_model1,best_model2,test_loader,best_threshold,n_steps_out):
    labels = []
    predictions = []
    with torch.no_grad():
        best_model1.eval()
        best_model2.eval()
        for batch_idx, (data1, target1, data2, target2) in enumerate(test_loader):
            n = data1.shape[0]
            preds = []
            for step_idx in range (n):
                data = data1[step_idx].unsqueeze(dim=0).to(device)
                # Forward pass
                ZNZ_output = best_model1(data).cpu().detach().numpy()
                ZNZ_output=np.where(ZNZ_output<best_threshold,0,ZNZ_output)
                ZNZ_output=np.where(ZNZ_output>best_threshold,1,ZNZ_output)
                data = torch.hstack((data2[step_idx].unsqueeze(dim=0),torch.tensor(ZNZ_output)))
                data = data.to(device)
                # Forward pass
                output = best_model2(data)
                pred=output.cpu().detach() * ZNZ_output
                if step_idx<n-1:
                    data1,data2 = test_generate_data(data1,ZNZ_output,data2,pred,step_idx)
                preds.append(pred.ravel())
            if (n==n_steps_out):
                labels.extend(target2.ravel())
                predictions.extend(preds)
    return (labels,predictions)

# Regression Network

In [17]:
class RegNetwork(nn.Module):
    def __init__(self,RNN , RNN_type , RNN_size , RNN_mean , RNN_sigma , NN_number , NN_size , NN_mean , NN_sigma ,
                 dropprob , n_features):
        super(RegNetwork,self).__init__()
        self.RNN=RNN
        self.RNN_type=RNN_type
        self.RNN_size=RNN_size
        self.RNN_mean=RNN_mean        
        self.RNN_sigma=RNN_sigma
        self.NN_number=NN_number
        self.NN_mean=NN_mean
        self.NN_sigma=NN_sigma
        self.dropprob=dropprob
        self.input_channels=n_features
        self.FC_size=NN_size
        if self.RNN:
            if RNN_type=='LSTM':
                self.rnn = nn.LSTM(self.input_channels, RNN_size, num_layers=1, bidirectional=False).to(device)
                self.input_channels= RNN_size
            elif RNN_type=='BiLSTM':
                self.rnn = nn.LSTM(self.input_channels, RNN_size, num_layers=1, bidirectional=True).to(device)
                self.input_channels= 2*RNN_size
            elif RNN_type=='GRU':
                self.rnn = nn.GRU(self.input_channels, RNN_size, num_layers=1, bidirectional=False).to(device)
                self.input_channels= RNN_size
            elif RNN_type=='BiGRU':
                self.rnn = nn.GRU(self.input_channels, RNN_size, num_layers=1, bidirectional=True).to(device)
                self.input_channels= 2*RNN_size

            for layer_p in self.rnn._all_weights:
                for p in layer_p:
                    if 'weight' in p:
                        torch.nn.init.normal_(self.rnn.__getattr__(p),mean=self.RNN_mean,std=self.RNN_sigma)
        
        self.fc1=nn.Linear(self.input_channels,self.FC_size)
        torch.nn.init.normal_(self.fc1.weight,mean=self.NN_mean,std=self.NN_sigma)
        torch.nn.init.normal_(self.fc1.bias,mean=self.NN_mean,std=self.NN_sigma)
        self.classifier1 = nn.Sequential(
            self.fc1,
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropprob, inplace=False))
        if (self.NN_number>1):
            self.fc2=nn.Linear(self.FC_size,self.FC_size)
            torch.nn.init.normal_(self.fc2.weight,mean=self.NN_mean,std=self.NN_sigma)
            torch.nn.init.normal_(self.fc2.bias,mean=self.NN_mean,std=self.NN_sigma)
            self.classifier2 = nn.Sequential(
                self.fc2,
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropprob, inplace=False))
        if (self.NN_number>2):
            self.fc3=nn.Linear(self.FC_size,self.FC_size)
            torch.nn.init.normal_(self.fc3.weight,mean=self.NN_mean,std=self.NN_sigma)
            torch.nn.init.normal_(self.fc3.bias,mean=self.NN_mean,std=self.NN_sigma)
            self.classifier3 = nn.Sequential(
                self.fc3,
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropprob, inplace=False))
        self.fc_f=nn.Linear(self.FC_size,1)
        torch.nn.init.normal_(self.fc_f.weight,mean=self.NN_mean,std=self.NN_sigma)
        torch.nn.init.normal_(self.fc_f.bias,mean=self.NN_mean,std=self.NN_sigma)
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
        
    def forward(self,x):
        if self.RNN:
            x=x.permute(1,0,2)
            output, _ = self.rnn(x)
            if self.RNN_type=='BiLSTM' or self.RNN_type=='BiGRU':
                Normal_RNN=output[-1, :, :self.RNN_size]
                Rev_RNN=output[0, :, self.RNN_size:]
                x = torch.cat((Normal_RNN, Rev_RNN), 1)
                x = F.relu(x)
                x = self.dropout(x)
            else:
                x = output[-1, :, :]
                x = F.relu(x)
                x = self.dropout(x)
                
        x=self.classifier1(x)
        if (self.NN_number>1):
            x=self.classifier2(x)
        
        if (self.NN_number>2):
            x=self.classifier3(x)
        
        predict = torch.sigmoid(self.fc_f(x))
        return (predict)

In [18]:
def Calibration3(RNN,RNN_type,calib_loader,valid_loader,n_features):
    best_MSE = 100
    if verbose:
        print('Training on ',device)
    RNN_hidden_size_list = [20, 50, 80, 100]
    dropoutList = [0, 0.15, 0.3, 0.45]
    weight_decay_list=[10**-5,10**-4,10**-3,10**-2]
    layer_number_list=[2,3]
    layer_size_list = [32, 64]
    learning_rate_list = [10**-5,10**-4,10**-3,10**-3]
    max_learning_steps = 400
    optimizers =['adam','sgd']
    max_num_models = 40
    for number in range(max_num_models):
        if verbose:
            print('model {0} out of {1}'.format(number+1,max_num_models))
        # hyper-parameters
        RNN_hidden_size = random.choice(RNN_hidden_size_list)
        dropprob = random.choice(dropoutList)
        RNN_mean=random.choice([-1,1])*logsampler(10 ** -4, 1)
        RNN_sigma = logsampler(10 ** -4, 1)
        Weight_decay=random.choice(weight_decay_list)
        optimizer_ = random.choice(optimizers)
        if RNN:
            layer_number=1
        else: 
            layer_number=random.choice(layer_number_list)
        layer_size = random.choice(layer_size_list)
        learning_rate=random.choice(learning_rate_list)
        NN_mean=random.choice([-1,1])*logsampler(10 ** -4, 1)
        N_sigma = logsampler(10 ** -4, 1)
        model_MSE = []
        
        #init model
        model = RegNetwork(RNN,RNN_type,RNN_hidden_size,RNN_mean,RNN_sigma,layer_number,layer_size,NN_mean,N_sigma,
                           dropprob,n_features).to(device)
        if optimizer_ == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=Weight_decay)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=Weight_decay)
        #training
        learning_steps = 0
        while learning_steps <= max_learning_steps:
            loss_per_epoch = 0
            model.train()
            for batch_idx, (data, target) in enumerate(calib_loader):
                data = data.to(device)
                target = target.to(device)
                # Forward pass
                output = model(data)
                criterion = nn.MSELoss()
                loss = criterion(output,target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_per_epoch+=loss.item()
            learning_steps += 1
            if learning_steps % 100 == 0:
                if verbose:
                    print("model trained for {0} epochs, loss per epoch is {1}.".format(learning_steps,loss_per_epoch/(batch_idx+1)))
                with torch.no_grad():
                    model.eval()
                    mse = []
                    for batch_idx, (data, target) in enumerate(valid_loader):
                        data = data.to(device)
                        target = target.to(device)
                        # Forward pass
                        output = model(data)
                        pred = output.cpu().detach().numpy().reshape(output.shape[0])
                        labels = target.cpu().numpy().reshape(output.shape[0])
                        if output.shape[0] > 60:
                            mse.append(metrics.mean_squared_error(labels, pred))
                    MSE=np.mean(mse)
                    if MSE < best_MSE:
                        best_MSE = MSE
                        best_learning_steps = learning_steps
                        best_LearningRate = learning_rate
                        best_dropprob = dropprob
                        best_layer_number= layer_number
                        best_layer_size= layer_size
                        best_NN_mean=NN_mean
                        best_N_sigma=N_sigma
                        best_RNN_hidden_size= RNN_hidden_size
                        best_RNN_mean=RNN_mean
                        best_RNN_sigma=RNN_sigma
                        best_weight_decay=Weight_decay
                        best_optimizer=optimizer_
    
    if verbose:
        print('best_MSE=', best_MSE)
        print('best_learning_steps=', best_learning_steps)
        print('best_LearningRate=', best_LearningRate)
        print('best_dropprob=', best_dropprob)
        print('best_weight_decay=',best_weight_decay)
        print('best_layer_number=', best_layer_number)
        print('best_layer_size=', best_layer_size)
        print('best_NN_mean=', best_NN_mean)
        print('best_N_sigma=', best_N_sigma)
        print('best_RNN_hidden_size=', best_RNN_hidden_size)
        print('best_RNN_mean=', best_RNN_mean)
        print('best_RNN_sigma=', best_RNN_sigma)
        print('best_optimizer=', best_optimizer)
    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_weight_decay':best_weight_decay,
                            'best_layer_number': best_layer_number,
                            'best_layer_size': best_layer_size, 
                            'best_NN_mean':best_NN_mean,
                            'best_N_sigma':best_N_sigma,
                            'best_RNN_hidden_size' :best_RNN_hidden_size,
                            'best_RNN_mean':best_RNN_mean,
                            'best_RNN_sigma': best_RNN_sigma,
                            'best_optimizer': best_optimizer}
    return best_hyperparameters

In [19]:
def Train_model3(RNN,RNN_type,best_hyperparameters,train_loader,n_features):
    best_learning_steps=best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_weight_decay=best_hyperparameters['best_weight_decay']
    best_layer_number=best_hyperparameters['best_layer_number']
    best_N_sigma=best_hyperparameters['best_N_sigma']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
    best_RNN_sigma=best_hyperparameters['best_RNN_sigma']
    best_RNN_mean=best_hyperparameters['best_RNN_mean']
    best_NN_mean=best_hyperparameters['best_NN_mean']
    best_optimizer=best_hyperparameters['best_optimizer']
    best_MSE=100

    for number_models in range(5):
        model = RegNetwork(RNN,RNN_type,best_RNN_hidden_size,best_RNN_mean,best_RNN_sigma,best_layer_number,
                           best_layer_size,best_NN_mean,best_N_sigma,best_dropprob,n_features).to(device)
        if best_optimizer == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(),lr=best_LearningRate,weight_decay=best_weight_decay)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr=best_LearningRate,weight_decay=best_weight_decay)
        learning_steps=0
        model.train()
        while (learning_steps<=best_learning_steps):    
            for batch_idx, (data, target) in enumerate (train_loader):
                data = data.to(device)
                target = target.to(device)
                # Forward pass
                output = model(data)
                L=F.mse_loss(target,output)
                optimizer.zero_grad()
                L.backward()
                optimizer.step()
            learning_steps+=1
        with torch.no_grad():
            model.eval()
            mse=[]
            for batch_idx, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)
                # Forward pass
                output = model(data)
                pred=output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if (output.shape[0]>30):
                    mse.append(metrics.mean_squared_error(labels, pred))
            MSE_Training=np.mean(mse)     
            if verbose:
                print('MSE on training data for model ',number_models+1,' = ',MSE_Training)
            if (MSE_Training<best_MSE):
                best_MSE=MSE_Training
                best_model=model
    return best_model

In [20]:
def test_predict3(best_model,test_loader):
    with torch.no_grad():
        best_model.eval()
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            labels = target.cpu().numpy().reshape(output.shape[0])
    return (pred,labels)

# SES

In [21]:
def SES(ts,alpha=0.1):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    
    #level (a), periodicity(p) and forecast (f)
    f = np.full((cols),np.nan)
    
    # Initialization
    f[0] = d[0]
# Create all the t+1 forecasts
    for t in range(0,cols-1):        
        f[t+1] = alpha*d[t] + (1-alpha)*f[t]   
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Error":d-f})
    return df

In [22]:
def evaluate_SES(DATA_Micro,n1,n2):
    Opt_SES=[0,10000]
    for i in np.arange (0,1,0.001):
        SES_train=SES(DATA_Micro.Qty[n1:n2],alpha=i)
        if (np.sqrt(mean_squared_error(SES_train['Forecast'].values,DATA_Micro['Qty'][n1:n2].values))<Opt_SES[1]):
            Opt_SES[1]=np.sqrt(mean_squared_error(SES_train['Forecast'].values,DATA_Micro['Qty'][n1:n2].values))
            Opt_SES[0]=i
    return (Opt_SES[0])

# Croston

In [23]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
# Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [24]:
def evaluate_CR(DATA_Micro,n1,n2):
    Opt_CR=[0,100]
    for i in np.arange (0,1,0.001):
        Cr_train=Croston(DATA_Micro.Qty[n1:n2],alpha=i)
        if (np.sqrt(mean_squared_error(Cr_train['Forecast'].values[:-1],DATA_Micro['Qty'][n1:n2].values))<Opt_CR[1]):
            Opt_CR[1]=np.sqrt(mean_squared_error(Cr_train['Forecast'].values[:-1],DATA_Micro['Qty'][n1:n2].values))
            Opt_CR[0]=i
    return(Opt_CR[0])

# SBA

In [25]:
def SBA(ts,extra_periods=1,alpha=0.1,beta=0.1):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
# Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*q + (1-beta)*p[t]
            f[t+1] = (1-alpha/2)*a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [26]:
def evaluate_SBA(DATA_Micro,n1,n2):
    Opt_SBA=[0,0,100]
    for i in np.arange (0,1,0.01):
        for j in np.arange (0,1,0.01):
            SBA_train=SBA(DATA_Micro.Qty[n1:n2],alpha=i,beta=j)
            if (np.sqrt(mean_squared_error(SBA_train['Forecast'].values[:-1],DATA_Micro['Qty'][n1:n2].values))<Opt_SBA[2]):
                Opt_SBA[2]=np.sqrt(mean_squared_error(SBA_train['Forecast'].values[:-1],DATA_Micro['Qty'][n1:n2].values))
                Opt_SBA[1]=j
                Opt_SBA[0]=i
    return(Opt_SBA[0],Opt_SBA[1])

In [27]:
def segmentate_results(predictions,n_steps_out,change=True):
    n = len(predictions)
    for i in range (0,n+1,n_steps_out):
        if (change) and (i<n):
            predictions[i:i+n_steps_out]=[predictions[i]]*n_steps_out
    return (predictions[0:i])

# Generating results

In [4]:
def save_to_csv(product,n_obs,ADI,CV2,n_steps_in,n_steps_out,AUROC_HM,MASE_HM,MASE_NN,MASE_SES,MASE_Cr,MASE_SBA,MASE_RNN,
                MASE_Naive,MASE_ZF,RMSSE_HM,RMSSE_NN,RMSSE_SES,RMSSE_Cr,RMSSE_SBA,RMSSE_RNN,RMSSE_Naive,RMSSE_ZF,file='Forecast_results.csv'):
    header=['product','Samples','ADI','CV2','n_steps_in','n_steps_out','AUROC_HM','MASE_HM','MASE_NN','MASE_SES','MASE_Cr','MASE_SBA','MASE_RNN','MASE_Naive','MASE_ZF',
    'RMSSE_HM','RMSSE_NN','RMSSE_SES','RMSSE_Cr','RMSSE_SBA','RMSSE_RNN','RMSSE_Naive','RMSSE_ZF']
    dicti={
        'product':product,
        'Samples':n_obs,
        'ADI':ADI,
        'CV2':CV2,
        'n_steps_in':n_steps_in,
        'n_steps_out':n_steps_out,
        'AUROC_HM':AUROC_HM,
        'MASE_HM':MASE_HM,
        'MASE_NN':MASE_NN,
        'MASE_SES':MASE_SES,
        'MASE_Cr':MASE_Cr,
        'MASE_SBA':MASE_SBA,
        'MASE_RNN':MASE_RNN,
        'MASE_Naive':MASE_Naive,
        'MASE_ZF': MASE_ZF,
        'RMSSE_HM':RMSSE_HM,
        'RMSSE_NN':RMSSE_NN,
        'RMSSE_SES':RMSSE_SES,
        'RMSSE_Cr':RMSSE_Cr,
        'RMSSE_SBA':RMSSE_SBA,
        'RMSSE_RNN':RMSSE_RNN,
        'RMSSE_Naive':RMSSE_Naive,
        'RMSSE_ZF':RMSSE_ZF
        }
    # saving results
    try:
        df=pd.read_csv('Forecast_results.csv')
        new=False
    except:
        new=True
    if new:
        with open('Forecast_results.csv','w') as fd:
            writer = csv.writer(fd)
            writer.writerow(header)
            writer = csv.DictWriter(fd, fieldnames=header)
            writer.writerow(dicti)
    else:
        with open('Forecast_results.csv','a',newline='') as fd:
            writer = csv.DictWriter(fd, fieldnames=header)
            writer.writerow(dicti)

In [36]:
def main(product,n_steps_in=1,n_steps_out_list=[1],path="./gen data/"):  
    DATA=pd.read_csv(path+product+".csv", sep=';',encoding = "ISO-8859-1")

    #preprocess
    data = datasets(DATA,n_steps_in)
    
    #number of input lags, train and test samples
    Nlags = data.get_Nlags()
    n0 = data.get_n0()
    n1 = data.get_n1()
    
    #loss function coef
    w0 = data.get_w0()
    w1 = data.get_w1()
    
    #metric
    metric='PRC'
    
    #number of features
    n_features=5
    n_features2=3
    n_features3=5
    
    #model type and param
    RNN='LSTM'
    batch_size=64
    evaluate_performance=True
    
    #Deep2net
    calib_data,valid_data,train_data,test_data=data.get_deep2net_datasets()
    calib_dataset=dataset_load(calib_data)
    valid_dataset=dataset_load(valid_data)
    train_dataset=dataset_load(train_data)
    test_dataset=dataset_load(test_data)
    calib_loader = DataLoader(dataset=calib_dataset,batch_size=batch_size,shuffle=False)
    valid_loader = DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=False)
    train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
    best_hyperparameters=Calibration(RNN,w1,w0,calib_loader,valid_loader,n_features,metric)
    best_model,best_threshold=Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric)
    best_hyperparameters2=Calibration2(calib_loader,valid_loader,n_features2,best_model,best_threshold)
    best_model2=Train_model2(best_hyperparameters2,train_loader,n_features2,best_model,best_threshold)
    
    #RegNetwork-RNN
    calib_data1,valid_data1,train_data1,test_data1,scaler=data.get_reg_datasets()
    calib_dataset1=dataset_load1(calib_data1)
    valid_dataset1=dataset_load1(valid_data1)
    train_dataset1=dataset_load1(train_data1)
    test_dataset1=dataset_load1(test_data1)
    calib_loader1 = DataLoader(dataset=calib_dataset1,batch_size=batch_size,shuffle=False)
    valid_loader1 = DataLoader(dataset=valid_dataset1,batch_size=batch_size,shuffle=False)
    train_loader1 = DataLoader(dataset=train_dataset1,batch_size=batch_size,shuffle=False)
    test_loader1 = DataLoader(dataset=test_dataset1,batch_size=10000,shuffle=False)
    best_hyperparameters_RNN=Calibration3(True,RNN,calib_loader1,valid_loader1,n_features3)
    best_model_RNN=Train_model3(True,RNN,best_hyperparameters_RNN,train_loader1,n_features3)
    pred_RNN, labels_RNN = test_predict3(best_model_RNN,test_loader1)
    pred_RNN=scaler.inverse_transform([pred_RNN]).ravel()
    labels_RNN=scaler.inverse_transform([labels_RNN]).ravel()

    #RegNetwork-MLP
    best_hyperparameters_MLP=Calibration3(False,RNN,calib_loader1,valid_loader1,n_features3)
    best_model_MLP=Train_model3(False,RNN,best_hyperparameters_MLP,train_loader1,n_features3)
    pred_MLP, labels_MLP = test_predict3(best_model_MLP,test_loader1)
    pred_MLP=scaler.inverse_transform([pred_MLP]).ravel()
    
    #SES
    SES_test=SES(DATA.Qty[:],evaluate_SES(DATA,n0,n1))
    SES_pred_=SES_test['Forecast'][n1:]
    
    #Croston
    Cr_test=Croston(DATA.Qty[0:],alpha=evaluate_CR(DATA,n0,n1))
    Cr_pred_=Cr_test['Forecast'][n1:-1]
    
    #SBA
    Opt_SBA=evaluate_SBA(DATA,n0,n1)
    SBA_test=SBA(DATA.Qty[0:],alpha=Opt_SBA[0],beta=Opt_SBA[1])
    SBA_pred_=SBA_test['Forecast'][n1:-1]
    
    #Naive
    naiv_pred_=DATA['Qty'][n1-Nlags:-Nlags].values
    
    #Saving to CSV
    n_obs = data.get_n_obs()
    ADI = data.get_ADI()
    CV2 = data.get_CV2()
    Train_X = data.get_train_Y()
    Test_Y = data.get_test_Y()
    
    test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)
    AUROC_HM = test_predict(best_model,test_loader)
    
    for n_steps_out in n_steps_out_list:
        test_loader = DataLoader(dataset=test_dataset,batch_size=n_steps_out,shuffle=False)
        labels_HM,predictions_HM = test_predict2(best_model,best_model2,test_loader,best_threshold,n_steps_out)
        seg_Test_Y = segmentate_results(Test_Y,n_steps_out,change=False)
        MASE_HM = MASE(Train_X,seg_Test_Y,predictions_HM).item()
        RMSSE_HM = RMSSE(Train_X,seg_Test_Y,predictions_HM).item()
    
        if n_steps_out==1:
            pred_MLP = segmentate_results(pred_MLP,n_steps_out,change=False)
            MASE_NN = MASE(Train_X,seg_Test_Y,pred_MLP)
            RMSSE_NN = RMSSE(Train_X,seg_Test_Y,pred_MLP)
            pred_RNN = segmentate_results(pred_RNN,n_steps_out,change=False)
            MASE_RNN = MASE(Train_X,seg_Test_Y,pred_RNN)
            RMSSE_RNN = RMSSE(Train_X,seg_Test_Y,pred_RNN)
        else:
            MASE_NN,RMSSE_NN,MASE_RNN,RMSSE_RNN=0,0,0,0
    
    
        SES_pred = segmentate_results(list(SES_pred_),n_steps_out)
        MASE_SES = MASE(Train_X,seg_Test_Y,SES_pred)
        RMSSE_SES = RMSSE(Train_X,seg_Test_Y,SES_pred)

        Cr_pred = segmentate_results(list(Cr_pred_),n_steps_out)
        MASE_Cr = MASE(Train_X,seg_Test_Y,Cr_pred)
        RMSSE_Cr = RMSSE(Train_X,seg_Test_Y,Cr_pred)

        SBA_pred = segmentate_results(list(SBA_pred_),n_steps_out)
        MASE_SBA = MASE(Train_X,seg_Test_Y,SBA_pred)
        RMSSE_SBA = RMSSE(Train_X,seg_Test_Y,SBA_pred)

        if (Nlags<n_steps_out):
            naiv_pred = segmentate_results(naiv_pred_,n_steps_out)
            MASE_Naive = MASE(Train_X,seg_Test_Y,naiv_pred)
            RMSSE_Naive = RMSSE(Train_X,seg_Test_Y,naiv_pred)
        else:
            naiv_pred = segmentate_results(naiv_pred_,n_steps_out,change=False)
            MASE_Naive = MASE(Train_X,seg_Test_Y,naiv_pred)
            RMSSE_Naive = RMSSE(Train_X,seg_Test_Y,naiv_pred)
    
        ZF_pred = [0]*len(seg_Test_Y)
        MASE_ZF = MASE(Train_X,seg_Test_Y,ZF_pred)
        RMSSE_ZF = RMSSE(Train_X,seg_Test_Y,ZF_pred)
    
        save_to_csv(product,n_obs,ADI,CV2,n_steps_in,n_steps_out,AUROC_HM,MASE_HM,MASE_NN,MASE_SES,MASE_Cr,MASE_SBA,MASE_RNN,
                MASE_Naive,MASE_ZF,RMSSE_HM,RMSSE_NN,RMSSE_SES,RMSSE_Cr,RMSSE_SBA,RMSSE_RNN,RMSSE_Naive,RMSSE_ZF)

In [37]:
global device
global verbose 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
verbose = True

#n_forecast_ahead
n_steps_in = 30
n_steps_out_list = [1,7]
products_list = open('product_List.txt','r').read().strip().split('\n')
for product in products_list:
    start=time.time()
    if verbose:
        print("Opening '{0}.csv'".format(product))
    main(product,n_steps_in,n_steps_out)
    print('Model trained on product "%s" in %i minutes and %i seconds' %(str(product),int((time.time()-start)/60),float((time.time()-start)%60)))

Opening 'CETREMIDE GELOSE.csv'
Training on  cuda
model 1 out of 1
model trained for 100 epochs, loss per epoch is 0.8470074424037227.


KeyboardInterrupt: 